# Spectral Analysis

This example demonstrates how to perform spectral and cross-spectral analysis. 

This document shows a concise workflow using the `scipy.signal` module. To learn more about the functions and keyword arguments available in this package, read SciPy's [signal processing documentation](https://docs.scipy.org/doc/scipy/reference/signal.html).

This notebook will cover:

1. Periodograms
1. Smoothing
1. Cross Spectral Analysis
1. Coherence

In [1]:
import xarray as xr
import geocat.datafiles as gcd
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import numpy as np

## Periodogram 

A periodogram estimate is the square of the coefficients from the Fourier transform of one dimension of spectral data.

### Read in Data

We'll use the [`geocat-datafiles`](https://github.com/NCAR/geocat-datafiles) module to access the relevant datafile for this example: `SOI_Darwin.nc`

Then, we use [`xarray.open_dataset()`](http://xarray.pydata.org/en/stable/generated/xarray.open_dataset.html) function to read the data as an xarray dataset.

This data represents the Darwin Southern Oscillation Index between 1882 and 1998 using normalized sea level pressure.

In [ ]:
soi_darwin = xr.open_dataset(gcd.get('netcdf_files/SOI_Darwin.nc'))

soi_darwin

### Time Series Correction

Currently, our `time` coordinate is in units of months since January 1882, we'll use `pandas` to change this to the conventional `DatetimeIndex`.

In [ ]:
start_date = pd.Timestamp('1882-01-01')
dates = [
    start_date + pd.DateOffset(months=int(month))
    for month in soi_darwin.indexes['time']
]
datetime_index = pd.DatetimeIndex(dates)

datetime_index

In [4]:
soi = soi_darwin.DSOI
soi['time'] = datetime_index

### Periodogram

Now we can call [`scipy.signal.periodogram()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.periodogram.html).

You can specify a `window` argument for smoothing your spectrum before spectral analysis. Here is a [list of supported Scipy windows](https://docs.scipy.org/doc/scipy/reference/signal.windows.html). The default is `boxcar` which is a unit window equivalent to no tapering.

Set the detrending kwarg to `constant` to remove the mean from our data.

There is also a `scaling` keyword that defaults to `density` for computing the power spectral density with units of {math}`V^{2}/Hz`, with V being the units of the input array (here dimensionless), and `Hz` representing your frequency units (we'll use per month instead of per second). `spectral` is also supported and produces the squared magnitude spectrum with units of {math}`V^{2}`.

In [5]:
freq_soi, psd_soi = scipy.signal.periodogram(
    soi,
    fs=1,  # sample monthly
    window='tukey',
    detrend='constant',
)

### Smoothing

To provide statistical reliability, the periodogram estimates **must be smoothed**.

You can use the same [`scipy.signal.windows`](https://docs.scipy.org/doc/scipy/reference/signal.windows.html) as for the tapering. We'll chose a standard Hann window.

In [6]:
k = 3  # smoothing constant

window = scipy.signal.windows.hann(2 * k + 1)  # Hann Window
window = window / window.sum()  # Normalize window

# Apply the convolution
smoothed_psd = scipy.signal.convolve(psd_soi, window, mode='same')

### Plot

In [ ]:
plt.plot(freq_soi, smoothed_psd)
plt.xlabel('Frequency (Cycles / Month)')
plt.ylabel('Variance (Pressure$^2$ / Frequency)')
plt.title('Darwin Southern Oscillation Index (1882 - 1998) - Power Spectral Density')
plt.xlim([0, 0.5]);

## Cross-Spectral Analysis

For cross-spectral analysis, we'll create plots of the following quantities: cospectrum, quadtrature, phase, coherence squared, and the periodogram of each dimensional array.

### Read in Data

For our cross-spectral data we'll look at `SLP_DarwinTahitiAnom` which contains Darwin and Tahiti sea level pressure anomalies between 1935-1998.

This data will need the same time series adjustment as in the periodogram example above.

In [ ]:
slp_darwin = xr.open_dataset(gcd.get('netcdf_files/SLP_DarwinTahitiAnom.nc'))

# Time Series Correction
start_date = pd.Timestamp('1935-01-01')  # Months Since January 1935
dates = [
    start_date + pd.DateOffset(months=int(month))
    for month in slp_darwin.indexes['time']
]
datetime_index = pd.DatetimeIndex(dates)

slp_darwin['time'] = datetime_index
slp_darwin

In [ ]:
dslp = slp_darwin.DSLP  # Darwin
dslp

In [ ]:
tslp = slp_darwin.TSLP  # Tahiti
tslp

In [ ]:
dslp.plot(label='Darwin')
tslp.plot(label='Tahiti')
plt.legend()
plt.ylabel('Sea Level Pressure Anomalies [hPa]');

### Cross-Power Spectral Density

We will calculate cospectrum, quadtrature, and phase by leveraging [`scipy.signal.csd()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.csd.html) and computing its real (cospectrum) and imaginary (quadtrature) components, and calculating the angle between them (phase).

In [12]:
f, pxy = scipy.signal.csd(
    dslp,
    tslp,
    fs=1,  # monthly
    detrend='constant',
)  # remove mean

cospectrum = np.real(pxy)
quadrature = np.imag(pxy)

phase = np.angle(pxy, deg=True)

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(10, 5))

axs[0].plot(f, cospectrum)
axs[0].set_xlim([0, 0.5])
axs[0].set_xlabel('Frequency (Cycles / Month)')
axs[0].set_ylabel('Cospectrum')

axs[1].plot(f, quadrature)
axs[1].set_xlim([0, 0.5])
axs[1].set_ylim([-3, 3])
axs[1].set_xlabel('Frequency (Cycles / Month)')
axs[1].set_ylabel('Quadrature')

axs[2].plot(f, phase)
axs[2].set_xlim([0, 0.5])
axs[2].set_ylim([-200, 200])
axs[2].set_xlabel('Frequency (Cycles / Month)')
axs[2].set_ylabel('Phase')

fig.suptitle(
    'Darwin and Tahiti Sea Level Pressure Anomalies (1935-1998) - Cross-Spectral Analysis'
)
plt.tight_layout();

### Coherence Squared

Coherence is a measure of how similar two specta are, with regards to their frequency and phase offset relative to one another. 

For coherence squared, simply square the output from [`scipy.signal.coherence()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.coherence.html).

In [14]:
f, cxy = scipy.signal.coherence(
    dslp,
    tslp,
    fs=1,  # monthly
    detrend='constant',
)  # remove mean

co_sq = cxy**2

In [ ]:
plt.plot(f, co_sq)
plt.xlim([0, 0.5])
plt.ylim([0, 1])
plt.xlabel('Frequency (Cycles / Month)')
plt.ylabel('Coherence SQ')

plt.title('Darwin and Tahiti Sea Level Pressure Anomalies (1935-1998)')
plt.tight_layout();

### Peridogram and Smoothing

In [16]:
# Calculate Periodogram
freq_dslp, dslp_psd = scipy.signal.periodogram(
    dslp,
    fs=1,  # monthly
    detrend='constant',
)  # remove mean
freq_tslp, tslp_psd = scipy.signal.periodogram(tslp, fs=1, detrend='constant')

# Perform modified Daniel Smoothing
dslp_smoothed = scipy.signal.convolve(dslp_psd, window, mode='same')
tslp_smoothed = scipy.signal.convolve(tslp_psd, window, mode='same')

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 5))

axs[0].plot(freq_dslp, dslp_smoothed)
axs[0].set_xlim([0, 0.5])
axs[0].set_ylim([0, 30])
axs[0].set_xlabel('Frequency (cycles/month)')
axs[0].set_ylabel('Variance of Darwin SLP')

axs[1].plot(freq_tslp, tslp_smoothed)
axs[1].set_xlim([0, 0.5])
axs[1].set_ylim([0, 30])
axs[1].set_xlabel('Frequency (cycles/month)')
axs[1].set_ylabel('Variance of Tahiti SLP')

fig.suptitle(
    'Darwin and Tahiti Sea Level Pressure (SLP) Anomalies (1935-1998) - Power Spectral Density'
)
plt.tight_layout();